### Create a Service to transcript the earnings call
The whisper service has already transcribed sound files relating to the earnings calls.  It was hosted on a snowpark container and processed each file with a medium sized GPU.  Whisper leverages pytorch (a deap leaning framework) to parse and transcribe the text.  If you want to set this up yourself, follow the instructions here to set up the container in Snowflake.

- git hub https://github.com/michaelgorkow/scs_whisper, 
- blog post https://github.com/michaelgorkow/scs_whisper

####  Listen to the calls that were transcribed

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


files = session.sql('''SELECT RELATIVE_PATH, GET_PRESIGNED_URL('@DOCUMENT_AI.EARNINGS_CALLS',RELATIVE_PATH) URL FROM DIRECTORY (@DOCUMENT_AI.EARNINGS_CALLS)''')


select_call = st.selectbox('Select Call:', files.select('RELATIVE_PATH'))

URL = files.filter(col('RELATIVE_PATH') == select_call).select('URL').collect()[0][0]
st.audio(URL, format="audio/mpeg")

#### The sound files have already been transcribed using the whisper service.
We will now create a table based on the raw results produced by the Whisper Service which was hosted in Snowflake.

In [ ]:
CREATE TABLE if NOT EXISTS DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT AS 

SELECT * FROM ORGDATACLOUD$INTERNAL$TRANSCRIPTS_FROM_EARNINGS_CALLS.TRANSCRIPTS.EARNINGS_TRANSCRIPTS;

SELECT * FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT

### Transform the transcript table

In [ ]:
CREATE TABLE IF NOT EXISTS DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS AS

SELECT RELATIVE_PATH, PARSE_JSON(TRANSCRIPT):language::TEXT LANGUAGE,

VALUE:end::FLOAT TIME_SECONDS,  
VALUE:text::TEXT TEXT 
FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT,
LATERAL FLATTEN (PARSE_JSON(TRANSCRIPT):segments);

SELECT * FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS LIMIT 5

#### Add Sentiment scores to the calls

In [ ]:
SELECT *, SNOWFLAKE.CORTEX.SENTIMENT(TEXT) FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS

#### Put all together in Streamlit

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

def sentiment(text):
    return call_function('snowflake.cortex.sentiment',text)

transcript_with_sentiment = session.table('DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS').with_column('sentiment',sentiment(col('TEXT')))

st.markdown('#### Calls with Sentiment')


st.dataframe(transcript_with_sentiment)
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='TIME_SECONDS',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

In [ ]:
grouped = transcript_with_sentiment.with_column('TIME',time_from_parts(15,0,'TIME_SECONDS')).\
with_column('MINUTES',date_trunc('minute','TIME'))
grouped = grouped.with_column('MINUTES',minute('MINUTES'))
data_grouped_minutes = grouped.group_by('RELATIVE_PATH','MINUTES').agg(array_agg('TEXT').alias('TEXT'),avg('SENTIMENT').alias('SENTIMENT'))

st.markdown('''Data Grouped to Minutes''')
data_grouped_minutes


In [ ]:
st.markdown('#### Sentiment Analysis during the duration of the last 3 quarterly earnings calls')
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='MINUTES',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

st.markdown(f'''**:bulb: Most positive minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').desc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

st.markdown(f'''**:warning: Most negative minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').asc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

In [ ]:
grouped_text = data_grouped_minutes.with_column('TEXT',replace(replace(replace(cast('TEXT',StringType()),'"',''),'[',''),']',''))
grouped_text

#### Save data in a table

In [ ]:
grouped_text.write.mode("overwrite").save_as_table("DEFAULT_SCHEMA.summary_text")